<a href="https://colab.research.google.com/github/zahraDehghanian97/LSTM_sequence_prediction/blob/master/LSTM_5_len.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **prerequisit**

In [22]:
%tensorflow_version 2.x
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LSTM , Dropout , Masking
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.backend import clear_session 
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping
from dataclasses import make_dataclass
import keras
# constant
num_class = 7736
dimension = 1
seq_len = 5

# **load data**

In [2]:

data_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_next_game/train_Copy.csv')
X = []
y = []
seq_lens = []
seq_lens_test = []

for d in data_train.values:
    temp = []
    for t in d[1].split(" "):
        temp.append(int(t))
    X.append((temp))
    y.append( d[2])
    seq_lens.append(len(temp))

data_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_next_game/test.csv')
x_test = []
index_test = []
for d in data_test.values:
    index_test.append(d[0])
    temp = []
    for t in d[1].split(" "):
        temp.append(int(t))
    x_test.append((temp))
    seq_lens_test.append(len(temp))
    
print(len(seq_lens))


499


# **convert to min size sequence**

In [10]:
X_test =np.full((len(seq_lens_test), seq_len, dimension), fill_value=0)
size_train = np.sum(seq_lens) - ((seq_len-1) * len(seq_lens) )
size_train += np.sum(seq_lens_test) - ((seq_len) * len(seq_lens_test))
X_train = np.full((size_train, seq_len, dimension), fill_value=0)
Y_train = np.full(size_train,fill_value=0)

# make train data in shape of array with len = seq_len
counter = 0
for i in range(len(seq_lens)):
  for j in range(seq_lens[i]-seq_len+1):
    xx = np.array(X[i][j:j+seq_len])
    X_train[counter, :, :] = xx.reshape(seq_len,1)
    if j == seq_lens[i]-seq_len  :
      Y_train[counter] = y[i]
    else :
      Y_train[counter] = (X[i][j+seq_len])
    counter +=1

# make tesr data in shape of array with len = seq_len
## note : use additional test data to improve training
counter_test = 0
for i in range(len(seq_lens_test)):
  for j in range(seq_lens_test[i]-seq_len+1):
    if j == seq_lens_test[i]-seq_len  :
      xx = np.array(np.array(x_test[i][j:j+seq_len]))
      X_test[counter_test, :, :] = xx.reshape(seq_len,1)
      counter_test += 1
    else :
      xx = np.array(x_test[i][j:j+seq_len])
      X_train[counter, :, :] = xx.reshape(seq_len,1)
      Y_train[counter] = (x_test[i][j+seq_len])
      counter += 1
      

print(X_test[0])
print(X_train[0])
print(Y_train[0])

2692
[[ 118]
 [3135]
 [ 683]
 [ 213]
 [ 298]]
[[   3]
 [  12]
 [ 262]
 [6094]
 [ 283]]
50


# **LSTM Model**

In [25]:
clear_session()
lstm_units = seq_len * 4

model = Sequential()
model.add(LSTM(lstm_units, return_sequences=True, input_shape=(seq_len,dimension)))
# model.add(Dropout(rate=0.3))
model.add(LSTM(lstm_units, return_sequences=True))
# model.add(Dropout(rate=0.3))
# model.add(LSTM(lstm_units, return_sequences=True))
# model.add(Dropout(rate=0.3))
model.add(LSTM(lstm_units))
# model.add(Dropout(rate=0.2))
# model.add(Dense(units=2 * num_class, activation='relu'))
model.add(Dense(units=num_class, activation='softmax'))
print(model.summary())

filepath="/content/drive/MyDrive/Colab Notebooks/predict_next_game/weights2.hdf5"
# load the trained LSTM model
# model.load_weights(filepath)
print("load saved model")

# callbacks
checkpoint = ModelCheckpoint(filepath, monitor='val_sparse_top_k_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
earlyStop = EarlyStopping(monitor='val_sparse_top_k_categorical_accuracy', patience=5)

# Fit the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=SparseCategoricalCrossentropy(), metrics=['sparse_top_k_categorical_accuracy'])
history = model.fit(X_train, Y_train, epochs=150,batch_size = 128,validation_split = 0.1,callbacks=[checkpoint,earlyStop])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 20)             1760      
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 20)             3280      
_________________________________________________________________
lstm_2 (LSTM)                (None, 20)                3280      
_________________________________________________________________
dense (Dense)                (None, 7736)              162456    
Total params: 170,776
Trainable params: 170,776
Non-trainable params: 0
_________________________________________________________________
None
load saved model
Epoch 1/150
434/434 [==============================] - 7s 8ms/step - loss: nan - sparse_top_k_categorical_accuracy: 4.3284e-04 - val_loss: nan - val_sparse_top_k_categorical_accuracy: 0.0000e+00

Epoch 00001: val_sparse_top_k_

# **Test**

In [ ]:
#compute hit rate for validation data
predicted_app = model.predict(Xpad_validation)
idx = (-predicted_app).argsort() 
hit = 0
for i in range(len(Xpad_validation)):
    if(y_validation[i]== idx[i,0] or y_validation[i]==idx[i,1] or y_validation[i]==idx[i,2] or y_validation[i]==idx[i,3] or y_validation[i]==idx[i,5] ):
        hit+=1 
print("hit rate = "+str(hit/len(Xpad_validation)))

# final result
predicted_app_test = model.predict(Xpad_test)
idx = (-predicted_app_test).argsort() 
# write to file
Point = make_dataclass("point", [("id", int), ("next_games", str)])
answer = []
for i in range(len(idx)):
  # proper format
  temp = ""
  for j in range(5):
    temp+= str(int(idx[i][j]))
    if j != 4 :
      temp += " "
  answer.append(Point(index_test[i],temp))
answer = pd.DataFrame(answer)
answer.to_csv('/content/drive/MyDrive/Colab Notebooks/predict_next_game/result.csv')


hit rate = 0.105
